# capacitated vehicle routing problem with time-windows

In [ ]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

import pandas as pd
from itertools import permutations
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [ ]:
METRICS_FACTOR = 1000
VEHICLE_NUM = 3
VEHICLE_CAPACITY = 50

## Define DATA

In [ ]:
# Make data
df = pd.DataFrame()
df['no'] = range(8)
df['coord'] = [(0,0), (1,1), (2,1), (1,0), (-1,-1), (-2,-1), (-1,0), (0,-1)]
df['x'] = df['coord'].apply(lambda x: x[0])
df['y'] = df['coord'].apply(lambda x: x[1])
df['demand'] = [0, 10, 10, 10, 10, 10, 10, 45]
df['time_window'] = [(0.0,100.0), (1.0,1.5), (2.0,3.5), (6.0,7.0), (0.0,10.0), (2.0,3.0), (0.0,10.0), (0,100.0)]
df['start_time'] = df['time_window'].apply(lambda x: x[0])
df['end_time'] = df['time_window'].apply(lambda x: x[1])
df['service_time'] = [0, 1, 2 ,3 , 2, 2, 2, 1]
df = df.drop(['coord', 'time_window'], axis=1)
df = df.sort_values('no')
df

In [ ]:
# Plot data
fig, ax = plt.subplots()
ax = df.plot.scatter(x='x', y='y', ax=ax, grid=True)
for i in df.index:
    no, x, y, d, st, en = df.loc[
        i, ['no', 'x', 'y', 'demand', 'start_time', 'end_time']]
    no, d = int(no), int(d)
    text = f'{no}\nd:{d}\ntw:{st}-{en}'
    ax.annotate(
        text, 
        xy=(x,y), 
        horizontalalignment='left', 
        verticalalignment='top', 
        color='C1', 
        alpha=0.5)
rng = (df[['x', 'y']].values.min(), df[['x', 'y']].values.max())
offset = (rng[1]-rng[0])*0.1
rng = (rng[0]-offset, rng[1]+offset)
ax.set_xlim(rng)
ax.set_ylim(rng)
fig.tight_layout()

## Make Model

In [ ]:
# Make data for model

data = {}
# distance_matrix
dists = pd.DataFrame(list(permutations(df['no'], 2)), columns=['from', 'to'])
coord_map = df.set_index('no')[['x', 'y']]
dists['diff_x'] = coord_map.loc[dists['from'], 'x'].values - coord_map.loc[dists['to'], 'x'].values
dists['diff_y'] = coord_map.loc[dists['from'], 'y'].values - coord_map.loc[dists['to'], 'y'].values
dists['distance'] = ((dists['diff_x']**2) + (dists['diff_y']**2))**(1/2)
data['distance_matrix'] = dists.pivot_table(index=['from'], columns=['to'], values=['distance'])
data['distance_matrix'] = data['distance_matrix'].fillna(0)
data['distance_matrix'] = (METRICS_FACTOR*data['distance_matrix']).astype(int)
data['distance_matrix'] = data['distance_matrix'].values.tolist()
# time_matrix
data['time_matrix'] = data['distance_matrix']
# time_windows
data['time_windows'] = (METRICS_FACTOR*df[['start_time', 'end_time']]).astype(int)
data['time_windows'] = data['time_windows'].values.tolist()
# demands
data['demands'] = df['demand'].values.tolist()
# service_time
data['service_time'] = (METRICS_FACTOR*df['service_time']).astype(int)
data['service_time'] = data['service_time'].values.tolist()
# num_vehicles
data['num_vehicles'] = VEHICLE_NUM
# vehicle_capacities
data['vehicle_capacities'] = [VEHICLE_CAPACITY]*VEHICLE_NUM
# depot
data['depot'] = 0

In [ ]:
# Set model
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [ ]:
# Set cost of total distance
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [ ]:
# Set constraint of capacity
def demand_callback(from_index):
    """Returns the demand of the node."""
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

In [ ]:
# Set constraint of time window
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes + service time of from_node."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node] + data['service_time'][from_node]

dim_name = 'Time'
time_callback_index = routing.RegisterTransitCallback(time_callback)
routing.AddDimension(
    time_callback_index,
    max(sum(data['time_windows'], [])),  # allow waiting time
    max(sum(data['time_windows'], [])),  # maximum time per vehicle
    False,  # Don't force start cumul to zero.
    dim_name)
time_dimension = routing.GetDimensionOrDie(dim_name)
# add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data['time_windows']):
    if location_idx == data['depot']:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
# add time window constraints for each vehicle start node.
depot_idx = data['depot']
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data['time_windows'][depot_idx][0],
        data['time_windows'][depot_idx][1])
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

### Main function

In [ ]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [ ]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

### Visualization

In [ ]:
# Display the routes.

def get_results(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    results = {}
    capa_dim = routing.GetDimensionOrDie('Capacity')
    time_dim = routing.GetDimensionOrDie('Time')
    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        result = {
            'route': [manager.IndexToNode(index)], 
            'cost': [routing.GetFixedCostOfVehicle(vehicle_id)],
            'capa': [0],                                  
            'time': [0]
            }
        while not routing.IsEnd(index):
            pre_index = index
            index = solution.Value(routing.NextVar(index))
            result['route'].append(manager.IndexToNode(index))
            result['cost'].append(
                routing.GetArcCostForVehicle(pre_index, index, vehicle_id))
            result['capa'].append(
                capa_dim.GetTransitValue(pre_index, index, vehicle_id))          
            result['time'].append(
                time_dim.GetTransitValue(pre_index, index, vehicle_id))
        result['cost'] = pd.Series(result['cost']).cumsum().to_list()
        result['capa'] = pd.Series(result['capa']).cumsum().to_list()
        result['time'] = pd.Series(result['time']).cumsum().to_list()
        results[vehicle_id] = result
    return results

solution_results = get_results(solution, routing, manager)
print('Result of Route: No.(cost,capacity,time)')
for i, res in solution_results.items():
  route = res['route']
  costs = res['cost']
  capas = res['capa']
  times = res['time']
  text = [f'{l}({c},{d}/{VEHICLE_CAPACITY},{t})' 
          for l, c, d, t in zip(route, costs, capas, times)]
  text = ' -> '.join(text)
  print(f'Vehicle {i}: {text}')
print('Objective value:', solution.ObjectiveValue())

In [ ]:
# Plot

# add point
fig, ax = plt.subplots()
ax = df.plot.scatter(x='x', y='y', ax=ax, grid=True)
for i in df.index:
    no, x, y, d, st, en = df.loc[
        i, ['no', 'x', 'y', 'demand', 'start_time', 'end_time']]
    no, d = int(no), int(d)
    text = f'{no}\nd:{d}\ntw:{st}-{en}'
    ax.annotate(
        text, 
        xy=(x,y), 
        horizontalalignment='left', 
        verticalalignment='top', 
        color='C1', 
        alpha=0.5)
rng = (df[['x', 'y']].values.min(), df[['x', 'y']].values.max())
offset = (rng[1]-rng[0])*0.1
rng = (rng[0]-offset, rng[1]+offset)
ax.set_xlim(rng)
ax.set_ylim(rng)

# add route
cmap = plt.get_cmap('jet', len(solution_results))
cood_map = df.set_index('no')[['x', 'y']]
for i, res in solution_results.items():
    route = res['route']
    froms = cood_map.loc[route].values[:-1]
    tos = cood_map.loc[route].values[1:]
    lines = [[f, t] for f, t in zip(froms, tos)]
    total_dist = res['cost'][-1]/METRICS_FACTOR
    lc = LineCollection(lines, color=cmap(i), label=f'{i}:{total_dist}')
    ax.add_collection(lc)
ax.legend()

fig.tight_layout()